In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [ ]:
data_dir = Path('../data')
data2_dir = Path('../data2')
output_dir = Path('../output')

ospiti_file = data2_dir / 'ospiti_cleared(6).csv'
episodi_file = data_dir / 'joe_rogan_episodes.csv'

output_file = output_dir / 'associazione_guest_visual_mean.csv'

df_ospiti = pd.read_csv(ospiti_file)

df_episodi = pd.read_csv(episodi_file)


In [ ]:
def normalizza_nome(nome):
    if pd.isna(nome):
        return ""
    nome = str(nome).lower().strip()
    nome = re.sub(r'[^a-zA-Z0-9\s]', '', nome)
    nome = re.sub(r'\s+', ' ', nome)
    return nome

df_ospiti['nome_normalizzato'] = df_ospiti['nome'].apply(normalizza_nome)
df_episodi['guest_normalizzato'] = df_episodi['guest'].apply(normalizza_nome)

In [ ]:
def nome_contenuto_in_guest(nome_ospite, nome_guest):
    if not nome_ospite or not nome_guest:
        return False
    
    return nome_ospite in nome_guest

ospiti_dati = {}

for idx, ospite in df_ospiti.iterrows():
    nome_ospite = ospite['nome_normalizzato']
    nome_originale = ospite['nome']
    
    episodi_ospite = df_episodi[df_episodi['guest_normalizzato'].apply(
        lambda x: nome_contenuto_in_guest(nome_ospite, x)
    )]
    
    if len(episodi_ospite) > 0:
        views_list = episodi_ospite['views'].tolist()
        likes_list = episodi_ospite['likes'].tolist()
        comments_list = episodi_ospite['comments'].tolist()
        
        views_media = np.mean(views_list)
        likes_media = np.mean(likes_list)
        comments_media = np.mean(comments_list)
        
        ospiti_dati[nome_originale] = {
            'nome': nome_originale,
            'views': views_media,
            'likes': likes_media,
            'comments': comments_media,
            'num_episodi': len(episodi_ospite)
        }
        

In [ ]:
if ospiti_dati:
    risultati = list(ospiti_dati.values())
    df_risultati = pd.DataFrame(risultati)
    
    df_risultati = df_risultati.sort_values('views', ascending=False)

    output_dir.mkdir(exist_ok=True)
    
    df_risultati_finale = df_risultati[['nome', 'views', 'likes', 'comments']].copy()
    
    df_risultati_finale['views'] = df_risultati_finale['views'].round(0).astype(int)
    df_risultati_finale['likes'] = df_risultati_finale['likes'].round(0).astype(int)
    df_risultati_finale['comments'] = df_risultati_finale['comments'].round(0).astype(int)
    
    df_risultati_finale.to_csv(output_file, index=False)
    
